# Intro to the popular Python package "Pandas"

This notebook covers the basics of manipulating data using Pandas.

**Authors**: Eric Kofman (ekofman@eng.ucsd.edu) and Pratibha Jagannatha (pjaganna@eng.ucsd.edu) 

**Credit**: Adapted from UCSD CMM262

# Import the pandas package. But nickname it 'pd' for short.

In [ ]:
import pandas as pd

# Read a CSV (or TSV or anything!)

Here we are reading in a .tsv where the first column is "geneSymbol" and the second column is "chromosome"

In [ ]:
pd.read_csv('public-data/1_programming/data/gene_chrom.tsv', sep='\t')

We want to save it to a variable so that we can manipulate it in other ways and CHECK IT OUT!

In [ ]:
gene_chrom_table = pd.read_csv('public-data/1_programming/data/gene_chrom.tsv', sep='\t')

# Check out your data! What are we working with here?

We can sneak a peek at just the first few rows...

In [ ]:
gene_chrom_table.head()

In [ ]:
gene_chrom_table.head(2)

Or the last few rows...

In [ ]:
gene_chrom_table.tail(3)

We can get a sense of the "shape" of the data...

In [ ]:
gene_chrom_table.shape

^ This is what that means:

In [ ]:
len(gene_chrom_table.columns)

In [ ]:
len(gene_chrom_table)

How about indexing into a row? 

In [ ]:
gene_chrom_table['geneSymbol']

In [ ]:
[gene_chrom_table.geneSymbol == 'TP53']

In [ ]:
gene_chrom_table[gene_chrom_table.geneSymbol == 'TP53']

You can also directly use the index number

In [ ]:
gene_chrom_table.loc[11939]

In [ ]:
gene_chrom_table[gene_chrom_table.geneSymbol.isin(['TP53', 'BRCA1'])]

# OK let's read in another file. This one is a bit trickier

This file, which can be found at ../data/chrom_lengths.tsv, is actually *space-separated* and does not yet have column labels.  

In [ ]:
chrom_length_table = pd.read_csv('public-data/1_programming/data/chrom_lengths.tsv', sep=' ', names=['chrom', 'length'])

In [ ]:
chrom_length_table.head()

But! We have a problem. It looks like our lengths have actually been processed by pandas as strings instead of integers when they were loaded from the file. 

In [ ]:
chrom_length_table.loc[0,'length']

In [ ]:
type(chrom_length_table.loc[0,'length'])

In [ ]:
type(249698942)

Python is **not** happy when we try to treat this value as an integer and use it in a summation

In [ ]:
chrom_length_table.loc[0, 'length'] + 1

# Apply magic

### So let's make a new column where we convert each of these values into integers so we can do fancy things like graph them. We're going to be using the "apply" construct, which basically means we are defining a function to process each row and do something to it, then return the result of all of those functions.

For each value, we're going to want to replace the commas in the string with an empty string...

In [ ]:
'249,698,942'.replace(',', '')

Then convert them to integers!

In [ ]:
int('249,698,942'.replace(',', ''))

In [ ]:
type(int('249,698,942'.replace(',', '')))

Et voila! We can treat them like numbers now.

In [ ]:
int('249,698,942'.replace(',', '')) + 58

### Okay so here's our function definition

In [ ]:
# Here's our function that will be "applied" to each row
def turn_string_number_into_integer(row):
    # Access the length from this row...
    string_length = row['length']
    # Convert the length into an integer...
    int_length = int(string_length.replace(',', ''))
    # And return the integer!
    return int_length
    

Now, when we "apply" the function to each row (note: axis=1 implies processing each row one at a time, otherwise the default is to process each column one at a time), we will be returned a new column with the function's results for each row!

In [ ]:
chrom_length_table.apply(turn_string_number_into_integer, axis=1)

And we have to make sure to save it back to our dataframe with a new name

In [ ]:
chrom_length_table['numeric_length'] = chrom_length_table.apply(turn_string_number_into_integer, axis=1)

In [ ]:
chrom_length_table.head()

Alright, we don't really need that old column anymore, do we? We can subset out just the columns we want to keep and save them into another dataframe object.

In [ ]:
chrom_length_table[['chrom', 'numeric_length']]

In [ ]:
new_chrom_length_table = chrom_length_table[['chrom', 'numeric_length']]

In [ ]:
new_chrom_length_table.head()

Here's a useful pandas function you apply to this column now: 

In [ ]:
new_chrom_length_table.numeric_length.mean()

Or you can get a plot of the lengths!

In [ ]:
new_chrom_length_table.numeric_length.plot(kind='bar')

### but surprise surprise, it turns out that Pandas actually has a parameter built in to account for string conversion of numbers...

In [ ]:
chrom_length_table = pd.read_csv('public-data/1_programming/data/chrom_lengths.tsv', sep=' ', names=['chrom', 'length'], thousands=',')
chrom_length_table.head()

# Gene Density

<img src="public-data/1_programming/img/geneDensityArticle2020.png">

Okay, so now we have a table of genes mapped to chromosomes (*chrom_length_table*), and a table telling us how long each chromosome is (*gene_chrom_table*).
Let's try to calculate **gene density per chromosome**. We will want to figure out the total number of genes on each chromosome normalized by how long that chromosome is

# Group by

A reminder of what our *gene_chrom_table* table looked like:

In [ ]:
gene_chrom_table.head()

In [ ]:
gene_chrom_table.groupby('chromosome').count()

In [ ]:
gene_count_by_chromosome = gene_chrom_table.groupby('chromosome').count()

In [ ]:
gene_count_by_chromosome.columns = ['count']

You can do other things besides count, for example calculate the mean, variance, or standard deviation by group. Check out the documentation here to learn what else is possible: https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html

So this is now a table where we have grouped by chromosome and gotten a total of how many rows are associated with that chromosome.

In [ ]:
gene_count_by_chromosome.head()

And this is our table specifying lengths per chromosome:

In [ ]:
chrom_length_table.head()

In [ ]:
### Now you try: get the mean, standard deviation, and variance of the length column in chrom_length_table

In [ ]:
### Now you try: return a new column with length divided by two from chrom_length_table

### We want to put information both about the chromosome lengths and their gene counts into the same table. So we want to *join* information from both tables together into one table. To do this, we need to make sure that both tables have indices in the same format that can be cross-referenced.

<img src="public-data/1_programming/img/chromNamesRant.png">

#### Anyways, we have to make sure that the chromosome names have the same format in both of our tables so we can combine them.

In [ ]:
def add_chr(r):
    return 'chr' + r['chrom']


In [ ]:
chrom_length_table['chromosome'] = chrom_length_table.apply(add_chr, axis=1)

In [ ]:
chrom_length_table.head()

### And now, we can join the two tables on the "chromosome" column!

In [ ]:
gene_count_by_chromosome.head()

In [ ]:
joined_table = chrom_length_table.join(gene_count_by_chromosome, on='chromosome')

In [ ]:
joined_table

# Column-wise operations

To calculate the gene density, we want to divide the number of genes by the number of bp in each chromosome. We can use pandas' column-wise operpations to do the job!

In [ ]:
joined_table['geneDensity'] = joined_table['count'] / joined_table['length']

In [ ]:
joined_table.head()

Let's all set the overall dataframe's index to be the chromosome name

In [ ]:
joined_table.index = joined_table['chromosome']

# This does the same thing as:

joined_table.index = joined_table.chromosome

In [ ]:
joined_table.head()

# And let's plot these densities now to get a sense of how gene density varies across the genome

In [ ]:
from matplotlib import pyplot as plt

plt.title('Gene Density per Chromosome')
plt.ylabel("Gene Density")
### Now you try: plot gene density as a bar plot. 

#### Confirmed: chromosome 19 has about double the number of genes per any given stretch of DNA as the rest of the genome